In [1]:
import pickle
from statistics import mean 
import collections

In [2]:
data = 'heloc'
method = 'kleor'

In [3]:
with open(data+'_'+method+'.pickle', 'rb') as f:
    results = pickle.load(f)

In [4]:
len(results)

5

In [5]:
sf_query = []
sf_nh_knn = []
sf_nun_knn = []
sf_nun = []
mahalanobis = []
sparsity = []
ood_distance = []
trust_score = []
lipschitz = []

In [6]:
for item in results:
    sf_query.append(item['sf_query'])
    sf_nun.append(item['sf_nun'])
    mahalanobis.append(item['mahalanobis'])
    sparsity.append(item['sparsity'])
    ood_distance.append(item['ood_distance'])
    trust_score.append(item['trust_score'])
    lipschitz.append(item['lipschitz'])

In [7]:
from sklearn.preprocessing import MinMaxScaler

# function to scale the values
def scale_values(input_list, feature_range=(0, 1)):
    # Reshape the input list to a 2D array for MinMaxScaler
    values_2d = [[value] for value in input_list]

    # Initialize the MinMaxScaler with the specified feature range
    scaler = MinMaxScaler(feature_range=feature_range)

    # Fit and transform the data using the scaler
    scaled_values = scaler.fit_transform(values_2d)

    # Extract the scaled values from the 2D array and return as a list
    scaled_list = [scaled_value[0] for scaled_value in scaled_values]

    return scaled_list

In [8]:
sf_query = [x for xs in sf_query for x in xs]
sf_nun = [x for xs in sf_nun for x in xs]
mahalanobis = [x for xs in mahalanobis for x in xs]
sparsity = [x for xs in sparsity for x in xs]
ood_distance = [x for xs in ood_distance for x in xs]
trust_score = [x for xs in trust_score for x in xs]
trust_score_sc = scale_values(trust_score)
lipschitz = [x for xs in lipschitz for x in xs]
lipschitz_sc = scale_values(lipschitz)

In [9]:
mean(lipschitz_sc)

0.001470535663587556

In [9]:
metric_zip = zip(sf_query, sf_nun, sparsity, ood_distance, trust_score_sc, lipschitz_sc)

In [10]:
unitary = []

# create unitary metric by combining the metrics in a relevant way
for sf_q, sf_n, sp, ood, ts, lips in metric_zip:
    
    if (sp == 0):
        val = sf_q + 0 + ts + (1-ood) + (1-lips)
    else:
        val = sf_q + (1/sp) + ts +(1-ood) + (1-lips)
    
    unitary.append(val)

In [11]:
unitary = mean(unitary)
unitary

2.6149737523535532

In [12]:
sf_query = mean(sf_query)
sf_query

0.44604295911352165

In [25]:
sf_nun = mean(sf_nun)
sf_nun

0.4206048313527957

In [26]:
mahalanobis = mean(mahalanobis)
mahalanobis

3.499647562683888

In [27]:
ood_distance = mean(ood_distance)
ood_distance

0.20347699166191374

In [28]:
# filter trust score
trust_score = [value for value in trust_score if value < 100]
trust_score = mean(trust_score)
trust_score

1.2716191231975553

In [29]:
lipschitz = mean(lipschitz)
lipschitz

575.3688119715665

In [30]:
sp = sparsity.copy()
sp = [1/val if val>0 else 0 for val in sp]
mean(sp)

0.06766399045620028

In [15]:
sparse = collections.Counter(sparsity)
sparse

Counter({17: 691,
         18: 635,
         13: 608,
         16: 730,
         11: 246,
         20: 129,
         15: 809,
         12: 393,
         19: 382,
         14: 770,
         9: 31,
         10: 123,
         8: 9,
         7: 2})

In [16]:
new_dict = {}
three = 0
one = 0
two = 0
for key in sparse:
    if key>=3:
        three += sparse[key]
    elif key == 1:
        one = sparse[key]
    elif key == 2:
        two = sparse[key]

new_dict['1'] = one
new_dict['2'] = two
new_dict['3+'] = three

new_dict

{'1': 0, '2': 0, '3+': 5558}

In [17]:
new_dict_p = {}

new_dict_p['1'] = (new_dict['1']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100
new_dict_p['2'] = (new_dict['2']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100
new_dict_p['3+'] = (new_dict['3+']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100

new_dict_p

{'1': 0.0, '2': 0.0, '3+': 100.0}